In [1]:
#引入包
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data

In [2]:
#加载数据，加载的在input_data.py里面
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# NETWORK TOPOLOGIES
n_hidden_1 = 256 #第一个隐藏层的神经元数
n_hidden_2 = 128 #第二个隐藏层的神经元数
n_input    = 784 #输入的像素 28*28
n_classes  = 10  #输出的label[0，0,0,0,0,1,0,0,0,0] one_hot
# INPUTS AND OUTPUTS
x=tf.placeholder("float",[None,n_input],name='x')
y=tf.placeholder("float",[None,n_classes],name='y')

#神经网络，设立2个隐藏，填充数据
#标准偏差
stddev=0.1
#权重
weights={
    "w1":tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev=stddev)),
    "w2":tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev=stddev)),
    "out":tf.Variable(tf.random_normal([n_hidden_2,n_classes],stddev=stddev))
}
#偏置值
biases={
    "b1":tf.Variable(tf.random_normal([n_hidden_1])),
    "b2":tf.Variable(tf.random_normal([n_hidden_2])),
    "out":tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READ")

NETWORK READ


In [4]:
#s激活函数，进行数据的操作 （BP算法）
def multilayer_perceotron(_X, _weights, _biases):
    #sigmoid激活函数
    layer_1=tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['w1']),_biases['b1']))#每次wx+b之后经过sigmoid函数激活
    layer_2=tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['w2']),_biases['b2']))
    return (tf.matmul(layer_2, _weights['out'])+_biases['out'])#返回的是10个输出

In [5]:
pred=multilayer_perceotron(x,weights,biases)

#损失函数 softmax_cross_entropy_with_logits中0.x版本和1.x不同的是1.x要加logits和labels
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
#执行GradientDescentOptimizer梯度下降以及minimize优化
optm=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
#如果相等返回true，不等返回flase
corr=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
#把true转为1，false为0，再相加，计算总的（正确率）
accr=tf.reduce_mean(tf.cast(corr,"float"))

#初始化
init=tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [8]:
#执行模型的训练
training_epochs = 100#迭代的次数
batch_size      = 100#每次处理的图片数
display_step    = 4#每次次打印一次
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
# 模型保存，持久化
saver=tf.train.Saver()
epoch=0
#for epoch in range(training_epochs):
while True:
    avg_cost = 0.   #平均误差
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION  训练集循环次数确定
    for i in range(total_batch):
        #去除x和y的值
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    #重新计算平均损失
    avg_cost = avg_cost / total_batch
    # DISPLAY  显示误差率和训练集的正确率以此测试集的正确率
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
        #saver.save(sess, './mn', epoch)
        if train_acc>0.9 and test_acc>0.9:
            saver.save(sess, './mn', epoch)
            break
    epoch+=1
print ("OPTIMIZATION FINISHED")
writer = tf.summary.FileWriter(r"C:\Users\ibf\Documents\code\tf",tf.get_default_graph())
writer.close()
#最后的损失函数下降的不是很多，正确率也不高，需要加大迭代次数

Epoch: 003/020 cost: 2.280538216
TRAIN ACCURACY: 0.100
TEST ACCURACY: 0.157
Epoch: 007/020 cost: 2.247892685
TRAIN ACCURACY: 0.240
TEST ACCURACY: 0.274
Epoch: 011/020 cost: 2.212539887
TRAIN ACCURACY: 0.460
TEST ACCURACY: 0.399
Epoch: 015/020 cost: 2.172422694
TRAIN ACCURACY: 0.440
TEST ACCURACY: 0.468
Epoch: 019/020 cost: 2.125489607
TRAIN ACCURACY: 0.520
TEST ACCURACY: 0.520
Epoch: 023/020 cost: 2.069665157
TRAIN ACCURACY: 0.500
TEST ACCURACY: 0.558
Epoch: 027/020 cost: 2.003387713
TRAIN ACCURACY: 0.590
TEST ACCURACY: 0.608
Epoch: 031/020 cost: 1.925586604
TRAIN ACCURACY: 0.640
TEST ACCURACY: 0.629
Epoch: 035/020 cost: 1.836715375
TRAIN ACCURACY: 0.640
TEST ACCURACY: 0.652
Epoch: 039/020 cost: 1.738751326
TRAIN ACCURACY: 0.680
TEST ACCURACY: 0.672
Epoch: 043/020 cost: 1.635182406
TRAIN ACCURACY: 0.670
TEST ACCURACY: 0.692
Epoch: 047/020 cost: 1.530344173
TRAIN ACCURACY: 0.710
TEST ACCURACY: 0.712
Epoch: 051/020 cost: 1.428311844
TRAIN ACCURACY: 0.720
TEST ACCURACY: 0.730
Epoch: 055/0